In [ ]:
"""
-------------------------------------------------------------------------
Name:            Joshua Navarro
Date:            4/22/2025
Project #:       Orbits Project
Status:          Completed
Class:           PHYS 2425
-------------------------------------------------------------------------
Objective:  
Simulate the orbits of Mercury and Earth around the Sun.
Track position, calculate orbital period, and analyze mechanical energy.
-------------------------------------------------------------------------
"""

import vpython as vp
import numpy as np
import matplotlib.pyplot as plt

# === CONSTANTS USED IN PHYSICS ===
G = 6.67430e-11  # Gravitational constant. Units: m^3/(kg·s^2)
Msun = 1.98855e30  # Mass of the Sun in kilograms
AU = 1.495978707e11  # Astronomical Unit: the average distance from Earth to the Sun in meters
year = 365 * 24 * 3600  # Number of seconds

#CREATE A 3D SCENE
scene = vp.canvas(title="Orbital Simulation of Mercury and Earth",
                  width=800, height=600, center=vp.vector(0, 0, 0),
                  background=vp.color.black)  # The space where planets will orbit in our simulation

#CREATE THE SUN
sun = vp.sphere(pos=vp.vector(0, 0, 0),  #Position at the center
                radius=0.1 * AU,  # Size (scaled for visibility)
                color=vp.color.orange,
                emissive=True)

scene.center = vp.vector(0, 0, 0) #Center the animation at the sun
scene.range = 1.5 * AU   #Set zoom range

#CREATE MERCURY
mercury_mass = 3.285e23# kg
mercury_r = 4.6e10 #distance from the Sun (perihelion)
mercury_v = 5.898e4 #meters per second
mercury = vp.sphere(pos=vp.vector(mercury_r, 0, 0), #Place it to the right of the Sun
                    radius=0.02 * AU, # Size (smallest planet)
                    color=vp.color.gray(0.5),
                    make_trail=True)
mercury.velocity = vp.vector(0, mercury_v, 0) # Moving straight up (tangent to orbit) vp.vector(x, y, z)

# CREATE EARTH
earth_mass = 5.972e24 # kg
earth_r = 1.4709e11 #meters from Sun (also at perihelion)
earth_v = 3.029e4 # meters per second
earth = vp.sphere(pos=vp.vector(earth_r, 0, 0), #Earth also starts to the right
                  radius=0.025 * AU, # Slightly bigger than Mercury
                  texture=vp.textures.earth, # Use realistic Earth image
                  make_trail=True)
earth.velocity = vp.vector(0, earth_v, 0)             # Also moving upward / vp.vector(x, y, z)

#TIME SETTINGS
t = 0 #Start time at 0 seconds
dt = 60 * 60 #Each step in the simulation is 1 hour

#ENERGY AND POSITION LIST
t_list = []
U_list, K_list, E_list = [], [], [] #Mercury: potential, kinetic, total energy
Ue_list, Ke_list, Ee_list = [], [], [] # Earth: same as above
mercury_Y_positions = []#Mercury’s vertical position over time
earth_Y_positions = [] #Earth: same as above

# MAIN SIMULATION LOOP
# This loop runs until we simulate 2 Earth years of time
while t < 2 * year:
    vp.rate(100)  # This slows the animation down in case the orbits starts jumping

    #MERCURY PHYSICS
    r_vec_mercury = mercury.pos - sun.pos # Vector from Sun to Mercury . updates the new vp.vector(x, y, z)
    r_mag_mercury = vp.mag(r_vec_mercury) # Distance between Sun and Mercury also vp.mag() Returns the magnitude (length) of a vector.
    r_hat_mercury = vp.norm(r_vec_mercury) # Direction from Sun to Mercury

    F_mercury = -(G * Msun * mercury_mass) / (r_mag_mercury ** 2) * r_hat_mercury     #F = -G * M1 * M2 / r^2 * direction
    a_mercury = F_mercury / mercury_mass            # a = F / m (Newton’s Second Law)

    # Update Mercury’s speed and position
    mercury.velocity += a_mercury * dt # New velocity
    mercury.pos += mercury.velocity * dt # New position

    # MERCURY ENERGY
    v_mag_mercury = vp.mag(mercury.velocity) # Speed of Mercury also vp.mag() Returns the magnitude (length) of a vector.
    U_mercury = -(G * Msun * mercury_mass) / r_mag_mercury # Gravitational potential energy
    K_mercury = 0.5 * mercury_mass * v_mag_mercury ** 2   # Kinetic energy
    E_mercury = K_mercury + U_mercury  # Total mechanical energy

    # Save energy and x and y positions to the lists for every step
    t_list.append(t)
    U_list.append(U_mercury)
    K_list.append(K_mercury)
    E_list.append(E_mercury)
    mercury_Y_positions.append(mercury.pos.y)

    #EARTH PHYSIC (SAME AS MERCURY)
    r_vec_earth = earth.pos - sun.pos
    r_mag_earth = vp.mag(r_vec_earth)
    r_hat_earth = vp.norm(r_vec_earth)
    F_earth = -(G * Msun * earth_mass) / (r_mag_earth ** 2) * r_hat_earth
    a_earth = F_earth / earth_mass
    earth.velocity += a_earth * dt
    earth.pos += earth.velocity * dt

    # === EARTH ENERGY ===
    v_mag_earth = vp.mag(earth.velocity)
    U_earth = -(G * Msun * earth_mass) / r_mag_earth
    K_earth = 0.5 * earth_mass * v_mag_earth ** 2
    E_earth = K_earth + U_earth

    # Save energy and position
    Ue_list.append(U_earth)
    Ke_list.append(K_earth)
    Ee_list.append(E_earth)
    earth_Y_positions.append(earth.pos.y)

    # Move to next time step
    t += dt

#Mercury Energy Graph
plt.figure()
plt.plot(t_list, U_list, label="Mercury U", color="blue")# Potential Energy
plt.plot(t_list, K_list, label="Mercury K", color="red") # Kinetic Energy
plt.plot(t_list, E_list, label="Mercury E", color="green")# Total Energy
plt.xlabel("Time (s)")
plt.ylabel("Energy (Joules)")
plt.title("Mercury Energy vs Time")
plt.legend(); plt.grid(); plt.show()

#Earth Energy Graph
plt.figure()
plt.plot(t_list, Ue_list, label="Earth U", color="blue")
plt.plot(t_list, Ke_list, label="Earth K", color="red")
plt.plot(t_list, Ee_list, label="Earth E", color="green")
plt.xlabel("Time (s)")
plt.ylabel("Energy (Joules)")
plt.title("Earth Energy vs Time")
plt.legend(); plt.grid(); plt.show()

#Vertical Position Graph
plt.figure()
plt.plot(t_list, mercury_Y_positions, label="Mercury y(t)", color="orange")
plt.plot(t_list, earth_Y_positions, label="Earth y(t)", color="blue")
plt.xlabel("Time (s)")
plt.ylabel("Y Position (m)")
plt.title("Planetary Y Position vs Time")
plt.legend(); plt.grid(); plt.show()